In [1]:
from dataprocessing.processors.refined_ingestion import TrackingDataRefinedProcess, TimetableRefinedProcess, BusStopRefinedProcess, haversine
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

In [2]:
# from dataprocessing.processors.sparketl import ETLSpark

# etl_spark = ETLSpark()

# df = etl_spark.sqlContext.read.parquet("/data/trusted/vehicles").withColumn("hour", F.hour(F.col("event_timestamp"))).sort(F.asc("event_timestamp"))

In [33]:
%%time
tracking_data = TrackingDataRefinedProcess(2020,5,3)

CPU times: user 42 ms, sys: 4.91 ms, total: 47 ms
Wall time: 407 ms


In [12]:
tracking_data.df = tracking_data.df.filter("line_code = 561 and vehicle = 'EA210'")

In [34]:
events_computed = tracking_data.compute_metrics()

In [14]:
stop_events = tracking_data.stop_events(events_computed)

In [15]:
event_edges =  tracking_data.event_edges(events_computed)

In [16]:
dff = tracking_data.process_events(stop_events,event_edges)

In [20]:
pdf.to_csv("sample_events.csv",index=False)

In [24]:
from dataprocessing.processors.sparketl import ETLSpark

etl_spark = ETLSpark()

df = etl_spark.sqlContext.read.option("header","true").csv("/data/refined/events")

In [25]:
df.show()

+---------+------------------+-------+-------------+-------------------+----------+----------+------------+----+----+-----+---+
|line_code|          line_way|vehicle|moving_status|    event_timestamp|  latitude| longitude|avg_velocity|hour|year|month|day|
+---------+------------------+-------+-------------+-------------------+----------+----------+------------+----+----+-----+---+
|      222|Terminal Boa Vista|  CI002|      STOPPED|2020-05-03 05:21:10|-25.382503|-49.196763|       14.46|   5|2020|    5|  3|
|      222|Terminal Boa Vista|  CI002|      STOPPED|2020-05-03 05:23:02|-25.384511|   -49.202|       10.47|   5|2020|    5|  3|
|      222|Terminal Boa Vista|  CI002|      STOPPED|2020-05-03 05:25:59|-25.388048|-49.211428|        9.34|   5|2020|    5|  3|
|      222|Terminal Boa Vista|  CI002|      STOPPED|2020-05-03 05:37:23|-25.393095|-49.241245|        5.76|   5|2020|    5|  3|
|      222|    Vila Esperanca|  CI002|      STOPPED|2020-05-03 05:42:24|-25.393053|-49.241345|        0.

In [26]:
pdf = df.toPandas()
pdf.to_csv("sample_events.csv",index=False)

In [31]:
df.select("line_code","vehicle").distinct().groupBy("line_code").agg(F.count("vehicle")).show(10)

+---------+--------------+
|line_code|count(vehicle)|
+---------+--------------+
|      870|             1|
|      307|             3|
|      475|             1|
|      205|             2|
|      272|             1|
|      462|             2|
|      470|             1|
|      711|             1|
|      635|             1|
|      625|             1|
+---------+--------------+
only showing top 10 rows



In [35]:
events_computed.select("line_code","vehicle").distinct().groupBy("line_code").agg(F.count("vehicle")).show(10)

+---------+--------------+
|line_code|count(vehicle)|
+---------+--------------+
|      870|             1|
|      307|             3|
|      475|             1|
|      205|             2|
|      272|             1|
|      462|             2|
|      470|             1|
|      711|             1|
|      635|             1|
|      625|             1|
+---------+--------------+
only showing top 10 rows

